# Airline AI Assistant
Build Multi-modal customer support agent for an airline with UI and function-calling.

In [2]:
from dotenv import load_dotenv
import os
import gradio as gr
from openai import OpenAI
import sqlite3

In [3]:
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
openai = OpenAI()

In [4]:
DB= "prices.db"
with sqlite3.connect(DB) as conn:
    cursor = conn.cursor()
    cursor.execute('CREATE TABLE IF NOT EXISTS PRICES (city TEXT PRIMARY KEY, price REAL)')
    conn.commit()

In [5]:
def set_ticket_price(city, price):
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute('INSERT INTO prices (city, price) VALUES (?,?) ON CONFLICT(city) DO UPDATE SET price =?',(city.lower(),price,price))
        conn.commit()

In [6]:
def get_ticket_price(city):
    print(f"Database tool called: Getting price for {city}")
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute('SELECT price FROM prices WHERE city =?',(city.lower(),))
        result = cursor.fetchone()
        return f'Ticket price for the {city} is ${result[0]}' if result else f"No price available for this city"

In [7]:
ticket_prices = {"london":799, "paris":899, "tokyo":1420, "sydney":2999}
for city, price in ticket_prices.items():
    set_ticket_price(city, price)

In [8]:
get_ticket_price('tokyo')

Database tool called: Getting price for tokyo


'Ticket price for the tokyo is $1420.0'

In [9]:
get_ticket_price('india')

Database tool called: Getting price for india


'No price available for this city'

In [10]:
def put_message_in_chatbot(message, history):
    return "", history + [{"role":"user","content":message}]

In [11]:
def talker(message):
    response = openai.audio.speech.create(
        model='gpt-4o-mini-tts',
        voice='onyx',
        input=message
    )
    return response.content

In [12]:
def artist(city):
    image_response = openai.images.generate(
        model ='gpt-image-1',
        prompt=f'An image of a vacation in the {city}, showing tourist spots and everything unique about {city}, in a vibrant pop-art style',
        size="1024x1024",
            n=1,
            response_format="b64_json",
    )
    image_base64 = image_response.data[0].b64_json
    image_data = base64.b64decode(image_base64)
    return Image.open(BytesIO(image_data))

In [13]:
def handle_tool_calls_and_return_cities(message):
    responses = []
    cities = []
    for tool_call in message.tool_calls:
        if tool_call.function.name == "get_ticket_price":
            arguments = json.loads(tool_call.function.arguments)
            city = arguments.get('destination_city')
            cities.append(city)
            price_details = get_ticket_price(city)
            responses.append({
                "role": "tool",
                "content": price_details,
                "tool_call_id": tool_call.id
            })
    return responses, cities

In [14]:
system_message = """
You are a helpful assistant for an Airline called FlightAI.
Give short, courteous answers, no more than 1 sentence.
Always be accurate. If you don't know the answer, say so.
"""

In [15]:
def chat(history):
    history = [{"role":h["role"], "content":h["content"]} for h in history]
    messages = [{"role": "system", "content": system_message}] + history
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
    cities = []
    image = None

    while response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        responses, cities = handle_tool_calls_and_return_cities(message)
        messages.append(message)
        messages.extend(responses)
        response = openai.chat.completions.create(model="gpt-4.1-mini", messages=messages, tools=tools)

    reply = response.choices[0].message.content
    history += [{"role":"assistant", "content":reply}]

    voice = talker(reply)

    if cities:
        image = artist(cities[0])
    
    return history, voice, image

In [ ]:
with gr.Blocks() as ui:
    with gr.Row():
        chatbot  = gr.Chatbot(height=500, type='messages')
        image_output = gr.Image(height=500, interactive=False)
    with gr.Row():
        audio_output = gr.Audio(autoplay=True)
    with gr.Row():
        message = gr.Textbox(label ="Chat with our AI Assistant")

    message.submit(put_message_in_chatbot, inputs=[message,chatbot],outputs=[message,chatbot]).then(
        chat, inputs=chatbot, outputs=[chatbot,audio_output,image_output]
    )
    ui.launch(debug=True)

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
